# 0 - Navigating Duramat API Example

**Requirements:**
- For private datasets, key to the Duramat API

**Objectives:**
Learn to navigate the various levels of the Duramat Datahub. Example updated from 
https://github.com/DuraMAT/DuraMAT-DataHub-API/blob/master/API_DuraMAT_DataHub_Walkthrough.ipynb


In [1]:
# if running on google colab, uncomment the next line and execute this cell to install the dependencies and prevent "ModuleNotFoundError" in later cells:
# pip install -r ../requirements.txt

In [1]:
import os
import pandas as pd
import json
import pprint as pp
import urllib.request as ur
import urllib.parse as up

In [2]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)


Working on a  Windows 10
Python version  3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]
Pandas version  2.1.0


In [3]:
# FILE TO READ IN DETAILS:
data_url = r'https://datahub.duramat.org/dataset/best-field-data/resource/1c3ac2d9-4734-49b5-b1e7-577a4cc62d80'
projectID = '8e40c3ae-f53e-4a48-8223-2d760840bf8f'
datasetID = '72334fe1-fa52-45f8-a43c-836a35b9c526'
filename = r'BEST_Data.csv'

In [4]:
# build url pieces
emn_datahub = 'https://datahub.duramat.org'

# Helper to build a URI for a given API action
action = lambda a: '{}/api/3/action/{}'.format(emn_datahub, a)

action('project_list')

'https://datahub.duramat.org/api/3/action/project_list'

In [5]:
# Set your API token
api_token = ''
request = ur.Request(action('project_list')) 


In [6]:
# Using only authorization for the GET header
request.add_header('Authorization', api_token)

In [7]:
response = ur.urlopen(request)
data = json.loads(response.read().decode('utf-8'))   
projects = data['result']
for proj in projects:
    print(proj['title'] + '\t' + proj['id'])

50-Year PV Modeling	11ee5f43-96d6-44e0-b257-e81bd3c4dccb
Accelerated Testing	09b00777-8372-4140-9cde-6cae5cad4c33
Adhesion and Adhesives	f00849eb-a9a8-4fdc-a6d4-46ca38af7571
Albedo Data for Bifacial PV Systems	cc870bb7-2478-4d50-bb85-45303ba9770e
Analyzing Hail Impacts on PV Modules Using Computational Simulation	cb533a4c-6c3c-48c7-bf36-6639ce092ece
BACKFLIP	b9727172-ff2e-4281-b021-a079aeca5255
Backside Defect Imaging in Si PV	a050c26c-772b-4a80-8fe0-6c29468d52f9
Backside UV Degradation Modelling	19ab4aab-76a9-4ada-af3d-e3467c804586
B and B Data	b629e1a7-78e3-4571-81fe-469d1977897d
BEST Field Degradation Research	8ce838c8-15ba-4787-807a-3efc3a4c0b0c
BS Depth Profile	894f62f1-997e-40da-ba60-8b1c0a9846c4
C-AST: Combined Accelerated Stress Testing	39478fc3-bb49-4803-b807-2b0c1bc47e86
Cell Crack Characterization using Digital Image Correlation	51db1d8d-668e-4e52-8252-ae1064c269e9
Cell Cracking and Power Loss in Modern Modules	3437b315-b17a-4b02-9d3e-ba67a0017e8c
Central Data Resource - Dat

In [8]:
# Make dictionary of parameters and encode them into uri string
params = {'permission': 'create_dataset'}
param_string = up.urlencode( params )
#Build composite url for request
url = action('project_list_for_user') + '?' + param_string
# https://datahub.duramat.org/api/3/action/project_list_for_user
request = ur.Request(url)
# Add authorization
request.add_header('Authorization', api_token)
response = ur.urlopen(request)
data = json.loads(response.read().decode('utf-8'))   
projects = data['result']
for proj in projects:
    print(proj['title'] + '\t' + proj['id'])

In [9]:
projects

[]

In [9]:
project_id = projectID
params = {
    'id': project_id,
    'include_datasets': True,
    'include_users': False
}
param_string = up.urlencode( params )
#Build composite url for request
url = action('project_show') + '?' + param_string
# https://datahub.duramat.org/api/3/action/project_show
request = ur.Request(url)
# Add authorization
request.add_header('Authorization', api_token)
response = ur.urlopen(request)
data = json.loads(response.read().decode('utf-8'))   
project_detail = data['result']
print (project_detail['title'] + '\t' + project_detail['id'])
datasets = project_detail['packages']
for dataset in datasets:
    print ('\t' + dataset['title'] + '\t' + dataset['id'])

NREL Bifacial Experimental Single-Axis Tracking (BEST) field 	8e40c3ae-f53e-4a48-8223-2d760840bf8f
	BEST Field Data	72334fe1-fa52-45f8-a43c-836a35b9c526
	BEST FIELD images	8cbc87c7-e1d3-49ce-82d5-634f7a429792


In [10]:
dataset_id ='72334fe1-fa52-45f8-a43c-836a35b9c526'  # For BEST Field Data
resource_id='1c3ac2d9-4734-49b5-b1e7-577a4cc62d80'

action = lambda a: '{}/api/3/action/{}'.format(emn_datahub, a)        
params = {
    'id': dataset_id,
}
param_string = up.urlencode( params )
#Build composite url for request
url = action('package_show') + '?' + param_string
# https://datahub.duramat.org/api/3/action/package_show
request = ur.Request(url)
# Add authorization
#request.add_header('Authorization', self.dhkey)
response = ur.urlopen(request)
dataset = json.loads(response.read().decode('utf-8'))   

# Find the resource with the specified resource_id
resource_url = None
for resource in dataset['result']['resources']:
    if resource['id'] == resource_id:
        resource_url = resource['url']
        break

if resource_url is None:
    raise ValueError(f'Resource with id {resource_id} not found in dataset {dataset_id}')

#Download the public CKAN url directly to pandas. Note! If you have the actual URL from browsing
#the data hub, you can simply place that resource download URL directly into a pandas read_csv() 
#call.
df = pd.read_csv(resource_url)
# Display the DataFrame
print(df.head())

            Unnamed: 0  row2dcp  row2Gpoa_front  row2kWh  row2dcv  row2dci  \
0  2019-06-01 00:15:00      0.0             0.0      0.0      0.0      0.0   
1  2019-06-01 00:30:00      0.0             0.0      0.0      0.0      0.0   
2  2019-06-01 00:45:00      0.0             0.0      0.0      0.0      0.0   
3  2019-06-01 01:00:00      0.0             0.0      0.0      0.0      0.0   
4  2019-06-01 01:15:00      0.0             0.0      0.0      0.0      0.0   

   row2tmod  Yf2  PR2  row8dcp  ...  Hydra_current_12  temp_ambient  \
0       NaN  0.0  0.0      0.0  ...               NaN  -6396.667036   
1       NaN  0.0  0.0      0.0  ...               NaN  -6396.572336   
2       NaN  0.0  0.0      0.0  ...               NaN  -6396.614104   
3       NaN  0.0  0.0      0.0  ...               NaN  -6396.775228   
4       NaN  0.0  0.0      0.0  ...               NaN  -6396.696048   

   wind_direction  wind_speed  SRRL Tower Dry Bulb Temp [deg C]  \
0             NaN         NaN        